# Data Engineer Associate Exam - Virtual Reality Fitness

ActiVR provides a virtual reality device designed for exercise and fitness.

ActiVR offers a range of products, including VR devices and subscription-based fitness programs through their apps.

The sales team at ActiVR wants to analyze user data to enhance their marketing strategy and evaluate their products. For this, it is crucial that the data is clean, accurate, and available for reporting.

They need your assistance in preparing the data before launching a new promotional campaign.


**Database Schema**

The data schema for ActiVR's database is outlined as follows:

- **events:** Contains records of events registered in different games.
- **games:** Stores information about various games available on the platform.
- **devices:** Holds data about the virtual reality devices used by the users.
- **users:** Contains details about the users utilizing the ActiVR platform.

![schema](schema.png)


# Task 1

ActiVR's sales team wants to use the information it has about users for targeted marketing.

However, they suspect that the data may need to be cleaned before.

The expected data format and types for the `users` table according to the sales team's requirements is shown in the table below.

Write an SQL query that returns the `users` table with the specified format. Ensure that your query does not modify the `users` table.


| Column Name     | Description                                                  |
|-----------------|--------------------------------------------------------------|
| user_id         | Unique integer (assigned by the database, cannot be altered). Missing values are not possible due to the database structure. |
| age             | Integer representing the age of the customer. Missing values should be replaced with the average age.                  |
| registration_date | Date when the user made an account first (YYYY-MM-DD). Missing values should be replaced with January 1st, 2024.   |
| email | Email address of the user. Missing values should be replaced with Unknown.  |
| workout_frequency        | Workout frequency as a lowercase string, one of: _minimal_, _flexible_, _regular_, _maximal_. Missing values must be replaced with _flexible_. |


In [12]:
SELECT
   user_id,
   COALESCE(age, (SELECT ROUND(AVG(age)) FROM users)) AS age,
   COALESCE(CAST(registration_date AS DATE),'2024-01-01') AS registration_date,
   COALESCE(email, 'Unknown') AS email,
   COALESCE(LOWER(workout_frequency), 'flexible') AS workout_frequency
FROM users;

user_id  age  ...                    email workout_frequency
0          1   56  ...         hi_1@example.com                  
1          2   46  ...      hello_2@myemail.com           minimal
2          3   32  ...        hello_3@email.com           maximal
3          4   60  ...          user4@email.com           maximal
4          5   25  ...           hi_5@email.com           minimal
..       ...  ...  ...                      ...               ...
345      346   48  ...  contact_346@example.com          flexible
346      347   52  ...    contact_347@email.com           minimal
347      348   50  ...   contact348@example.com                  
348      349   38  ...          hi349@email.com           regular
349      350   49  ...       hi_350@myemail.com           maximal

[350 rows x 5 columns]

In [13]:
SELECT * 
FROM users
WHERE age > 50; 

user_id  age registration_date                  email workout_frequency
0          1   56        2022-09-20       hi_1@example.com                  
1          4   60        2023-02-25        user4@email.com           Maximal
2          7   56        2020-12-12      hello_7@email.com           MAXIMAL
3         13   53        2023-10-17      user_13@email.com          Flexible
4         14   57        2022-04-27     user14@example.com          flexible
..       ...  ...               ...                    ...               ...
97       340   52        2021-12-26  contact_340@email.com          Flexible
98       341   58        2023-09-03     hi_341@example.com          Flexible
99       342   54        2023-02-06    user342@myemail.com           regular
100      345   63        2022-10-12    user345@example.com          flexible
101      347   52        2020-06-01  contact_347@email.com           Minimal

[102 rows x 5 columns]

# Task 2
It seems like there are missing values in the `events` table for the column `game_id` for all events before the year 2021.

However, we know that before 2021 there were only games where the `game_type` is `running`. The `game_id` for these games can be found in the `games` table.

Write a query so that the `events` table has a `game_id` for all events including those before 2021.

In [14]:
SELECT
	e.event_id,
	COALESCE(e.game_id, g.game_id) AS game_id,
	e.device_id,
	e.user_id,
	e.event_time
FROM events e
LEFT JOIN games g
	ON e.game_id IS NULL
	AND g.game_type = 'running';

event_id  game_id  device_id  user_id          event_time
0            1        3          4       73 2021-06-11 02:07:04
1            2        3          5      141 2023-05-28 16:15:07
2            3        4          1       70 2023-08-31 13:28:50
3            4        4          2      262 2020-06-18 17:50:41
4            5        1          2      340 2021-01-21 06:34:48
...        ...      ...        ...      ...                 ...
1495      1496        4          1      225 2022-12-17 22:48:18
1496      1497        1          2      333 2022-04-19 03:50:09
1497      1498        1          3       42 2021-10-13 02:10:40
1498      1499        1          1      303 2022-07-01 09:38:06
1499      1500        2          5      246 2023-02-27 01:47:40

[1500 rows x 5 columns]

# Task 3

ActiVR's sales team plans to launch a promotion for upgrades to virtual reality devices.

They aim to target customers who have participated in events related to specific game types.

Write a SQL query to provide the `user_id` and `event_time` for users who have participated in events related to `biking` games.


In [15]:
SELECT
	e.user_id,
	e.event_time
FROM events e
JOIN games g
	ON e.game_id = g.game_id
WHERE g.game_type = 'biking';

user_id          event_time
0        340 2021-01-21 06:34:48
1         49 2021-10-17 08:53:15
2        109 2021-12-19 16:01:28
3        216 2023-03-16 14:57:29
4        339 2021-01-02 04:51:58
..       ...                 ...
293      266 2021-07-30 08:18:42
294      296 2023-02-28 10:16:18
295      333 2022-04-19 03:50:09
296       42 2021-10-13 02:10:40
297      303 2022-07-01 09:38:06

[298 rows x 2 columns]

# Task 4

After running their promotion, the sales team at ActiVR wants to investigate the results.

To do so, they require insights into the number of users who participated in events for each `game_type`.

Write a SQL query that returns the count of unique users for each game type `game_type` and `game_id`. The user count should be shown in a column `user_count`.

In [16]:
SELECT workout_frequency, COUNT(user_id) AS user_count
FROM users
GROUP BY workout_frequency; 

workout_frequency  user_count
0          Flexible          41
1                            18
2           MAXIMAL          30
3           Regular          38
4           maximal          35
5          flexible          37
6           minimal          41
7           Maximal          43
8           Minimal          32
9           regular          35